In [1]:
import re
import networkx as nx
from itertools import combinations

In [2]:
p = re.compile("Valve ([A-Z]{2}) has flow rate=(\d+); tunnel.? lead.? to valve.? ([A-Z ,]+)$")

with open("input", "r") as f:
    valve_net_raw = [p.match(line).groups() for line in f.readlines()]

In [3]:
G = nx.Graph()
G.add_edges_from((node, neighbour)
              for node, weight, neighbours in valve_net_raw
              for neighbour in neighbours.split(', '))

In [4]:
valve_net = {node: {'neighbours': neighbours.split(', '), 'capacity': int(capacity)}
             for node, capacity, neighbours in valve_net_raw}

In [5]:
start = 'AA'
to_open = [valve for valve, v in valve_net.items() if v['capacity'] != 0] + [start]
distances = {tuple(sorted(pair)): len(nx.shortest_path(G, *pair)) - 1
             for pair in combinations(to_open, 2)}

In [6]:
def pressure(to_open, valve, t, p):
    if t < 1:
        return p
    
    _to_open = to_open.copy()
    _to_open.pop(_to_open.index(valve))

    if valve != start:
        t -= 1
        p += valve_net[valve]['capacity'] * t
    
    if len(_to_open) == 0:
        return p
    
    return max(pressure(_to_open, v, t - distances[tuple(sorted((valve, v)))], p)
               for v in _to_open)

### Part 1

In [7]:
pressure(to_open, start, 30, 0)

2077

### Part 2

In [8]:
to_open = [valve for valve, v in valve_net.items() if v['capacity'] != 0]
_to_open_combinations = tuple(combinations(to_open, len(to_open) // 2))
_me_to_open = _to_open_combinations[:len(_to_open_combinations) // 2]

me_to_open = [list(_) + [start] for _ in _me_to_open]
elephant_to_open = [list(set(to_open) ^ set(_)) + [start] for _ in _me_to_open]

In [9]:
max(pressure(me, start, 26, 0) + pressure(elephant, start, 26, 0)
    for me, elephant in zip(me_to_open, elephant_to_open))

2741